# 📊 Darvas Box Momentum Analyzer

A comprehensive tool for identifying **Darvas Box** setups in NSE/BSE stocks.

---

## Overview

The Darvas Box strategy, developed by Nicolas Darvas, is a momentum-based trading system that:
1. Identifies stocks making new 52-week highs
2. Detects consolidation "boxes" after the high
3. Generates entry signals on breakouts with volume confirmation
4. Uses trailing stops based on box bottoms

## 1. Setup & Imports

In [ ]:
# Install required packages if not already installed
# !pip install yfinance pandas numpy plotly

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Configuration Constants
CONFIRMATION_DAYS = 3        # Days to confirm box top/bottom
VOLUME_MULTIPLIER = 1.5      # Volume must be 1.5x 20-day average for breakout
ENTRY_BUFFER = 0.001         # 0.1% above box top for entry
LOOKBACK_PERIOD = '1y'       # 1 year of data for 52-week high calculation
VOLUME_MA_PERIOD = 20        # Moving average period for volume

print("✅ Setup complete! Libraries loaded successfully.")

✅ Setup complete! Libraries loaded successfully.


## 2. Data Acquisition Module

In [3]:
def fetch_stock_data(symbol: str, period: str = '1y') -> Optional[pd.DataFrame]:
    """
    Fetch historical OHLCV data for a given stock symbol.
    
    Args:
        symbol: Stock ticker (e.g., 'RELIANCE.NS' for NSE, 'RELIANCE.BO' for BSE)
        period: Data period ('1y', '2y', etc.)
    
    Returns:
        DataFrame with OHLCV data or None if fetch fails
    """
    try:
        ticker = yf.Ticker(symbol)
        df = ticker.history(period=period)
        
        if df.empty:
            print(f"⚠️ No data found for {symbol}. Check if the ticker is valid.")
            return None
        
        # Clean column names
        df.columns = [col.lower() for col in df.columns]
        
        # Ensure we have required columns
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in df.columns for col in required_cols):
            print(f"⚠️ Missing required columns for {symbol}")
            return None
        
        # Reset index to make date a column
        df = df.reset_index()
        df.rename(columns={'Date': 'date', 'index': 'date'}, inplace=True)
        
        # Convert timezone-aware datetime to timezone-naive
        if df['date'].dt.tz is not None:
            df['date'] = df['date'].dt.tz_localize(None)
        
        print(f"✅ Loaded {len(df)} days of data for {symbol}")
        return df
        
    except Exception as e:
        print(f"❌ Error fetching data for {symbol}: {str(e)}")
        return None


# Test data acquisition
test_df = fetch_stock_data('RELIANCE.NS')
if test_df is not None:
    print(f"\nSample data preview:")
    display(test_df.tail())

✅ Loaded 251 days of data for RELIANCE.NS

Sample data preview:


,date,open,high,low,close,volume,dividends,stock splits
246,2025-12-19,1554.400024,1574.199951,1551.000000,1565.099976,13937275,0.0,0.0
247,2025-12-22,1573.500000,1577.500000,1565.300049,1575.400024,10184852,0.0,0.0
248,2025-12-23,1580.900024,1580.900024,1568.099976,1570.699951,7506564,0.0,0.0
249,2025-12-24,1572.699951,1575.699951,1553.599976,1558.199951,8815745,0.0,0.0
250,2025-12-26,1554.699951,1561.000000,1554.300049,1559.199951,624404,0.0,0.0


## 3. Darvas Box Detection Algorithm

In [4]:
class DarvasBox:
    """Represents a single Darvas Box."""
    def __init__(self, top: float, bottom: float, start_date, end_date=None):
        self.top = top
        self.bottom = bottom
        self.start_date = start_date
        self.end_date = end_date
        self.is_active = True
        self.breakout_date = None
        self.breakout_volume_confirmed = False
    
    def __repr__(self):
        return f"DarvasBox(top={self.top:.2f}, bottom={self.bottom:.2f}, start={self.start_date})"


def detect_darvas_boxes(df: pd.DataFrame, 
                        confirmation_days: int = 3,
                        volume_multiplier: float = 1.5) -> Tuple[List[DarvasBox], pd.DataFrame]:
    """
    Detect Darvas Boxes in the price data.
    
    Algorithm:
    1. Find 52-week highs
    2. If high is not broken for 'confirmation_days', it becomes Box Top
    3. After Box Top is set, find the low
    4. If low is not broken for 'confirmation_days', it becomes Box Bottom
    5. Validate breakouts with volume filter
    
    Args:
        df: DataFrame with OHLCV data
        confirmation_days: Days to confirm box top/bottom
        volume_multiplier: Required volume spike for valid breakout
    
    Returns:
        Tuple of (list of DarvasBox objects, enhanced DataFrame)
    """
    df = df.copy()
    boxes = []
    
    # Calculate 52-week high (rolling 252 trading days)
    df['52w_high'] = df['high'].rolling(window=252, min_periods=1).max()
    
    # Calculate 20-day average volume
    df['avg_volume'] = df['volume'].rolling(window=VOLUME_MA_PERIOD).mean()
    
    # State variables
    potential_top = None
    potential_top_date = None
    top_confirmation_count = 0
    
    box_top = None
    box_top_date = None
    
    potential_bottom = None
    potential_bottom_date = None
    bottom_confirmation_count = 0
    
    current_box = None
    
    # Initialize columns for tracking
    df['box_top'] = np.nan
    df['box_bottom'] = np.nan
    df['in_box'] = False
    df['breakout'] = False
    df['volume_confirmed'] = False
    
    for i in range(len(df)):
        row = df.iloc[i]
        current_high = row['high']
        current_low = row['low']
        current_close = row['close']
        current_volume = row['volume']
        current_date = row['date']
        avg_vol = row['avg_volume'] if pd.notna(row['avg_volume']) else current_volume
        
        # Check if we're at or near 52-week high (within 5%)
        is_near_52w_high = current_high >= row['52w_high'] * 0.95
        
        # === STATE MACHINE FOR BOX DETECTION ===
        
        # Phase 1: Looking for new high (potential box top)
        if potential_top is None or current_high > potential_top:
            if is_near_52w_high:
                potential_top = current_high
                potential_top_date = current_date
                top_confirmation_count = 0
                # Reset bottom search
                potential_bottom = None
                bottom_confirmation_count = 0
        else:
            # High not broken - increment confirmation
            if potential_top is not None:
                top_confirmation_count += 1
        
        # Phase 2: Box Top confirmed
        if top_confirmation_count >= confirmation_days and box_top is None:
            box_top = potential_top
            box_top_date = potential_top_date
            # Start looking for bottom
            potential_bottom = current_low
            potential_bottom_date = current_date
            bottom_confirmation_count = 0
        
        # Phase 3: Looking for box bottom (after top is confirmed)
        if box_top is not None and current_box is None:
            if potential_bottom is None or current_low < potential_bottom:
                potential_bottom = current_low
                potential_bottom_date = current_date
                bottom_confirmation_count = 0
            else:
                bottom_confirmation_count += 1
            
            # Box Bottom confirmed
            if bottom_confirmation_count >= confirmation_days:
                current_box = DarvasBox(
                    top=box_top,
                    bottom=potential_bottom,
                    start_date=box_top_date
                )
                boxes.append(current_box)
        
        # Phase 4: Inside a box - check for breakout or breakdown
        if current_box is not None and current_box.is_active:
            df.at[df.index[i], 'box_top'] = current_box.top
            df.at[df.index[i], 'box_bottom'] = current_box.bottom
            df.at[df.index[i], 'in_box'] = True
            
            # Breakout above box top
            if current_close > current_box.top:
                volume_confirmed = current_volume >= avg_vol * volume_multiplier
                current_box.breakout_date = current_date
                current_box.breakout_volume_confirmed = volume_confirmed
                current_box.end_date = current_date
                current_box.is_active = False
                
                df.at[df.index[i], 'breakout'] = True
                df.at[df.index[i], 'volume_confirmed'] = volume_confirmed
                
                # Reset for new box
                potential_top = current_high
                potential_top_date = current_date
                top_confirmation_count = 0
                box_top = None
                potential_bottom = None
                current_box = None
            
            # Breakdown below box bottom (stop-loss triggered)
            elif current_close < current_box.bottom:
                current_box.end_date = current_date
                current_box.is_active = False
                
                # Reset everything
                potential_top = None
                top_confirmation_count = 0
                box_top = None
                potential_bottom = None
                current_box = None
    
    return boxes, df


# Test box detection
if test_df is not None:
    boxes, enhanced_df = detect_darvas_boxes(test_df)
    print(f"\n📦 Found {len(boxes)} Darvas Boxes:")
    for i, box in enumerate(boxes[-5:], 1):  # Show last 5 boxes
        print(f"  Box {i}: Top={box.top:.2f}, Bottom={box.bottom:.2f}, Start={box.start_date.strftime('%Y-%m-%d')}")


📦 Found 8 Darvas Boxes:
  Box 1: Top=1433.77, Bottom=1407.08, Start=2025-05-05
  Box 2: Top=1435.27, Bottom=1408.67, Start=2025-05-26
  Box 3: Top=1544.83, Bottom=1376.70, Start=2025-07-09
  Box 4: Top=1508.30, Bottom=1470.10, Start=2025-10-29
  Box 5: Top=1524.70, Bottom=1517.60, Start=2025-11-12


## 4. Entry & Exit Signal Generation

In [5]:
def generate_signals(df: pd.DataFrame, boxes: List[DarvasBox], symbol: str) -> Dict:
    """
    Generate trading signals based on Darvas Box analysis.
    
    Args:
        df: Enhanced DataFrame with box information
        boxes: List of detected Darvas Boxes
        symbol: Stock symbol
    
    Returns:
        Dictionary with signal information
    """
    if not boxes:
        return {
            'symbol': symbol,
            'status': 'No Setup',
            'box_top': None,
            'box_bottom': None,
            'entry_price': None,
            'stop_loss': None,
            'current_price': df['close'].iloc[-1] if len(df) > 0 else None,
            'risk_reward': None,
            'volume_confirmed': False
        }
    
    # Get the most recent box
    latest_box = boxes[-1]
    current_price = df['close'].iloc[-1]
    latest_date = df['date'].iloc[-1]
    
    # Determine status
    if latest_box.is_active:
        status = 'Inside Box'
    elif latest_box.breakout_date is not None:
        days_since_breakout = (latest_date - latest_box.breakout_date).days
        if days_since_breakout <= 5 and latest_box.breakout_volume_confirmed:
            status = 'Breakout (Volume ✓)'
        elif days_since_breakout <= 5:
            status = 'Breakout (Low Volume)'
        else:
            status = 'Post-Breakout'
    else:
        status = 'Box Closed'
    
    # Calculate entry and stop-loss
    entry_price = latest_box.top * (1 + ENTRY_BUFFER)
    stop_loss = latest_box.bottom
    
    # Calculate risk-reward (assuming 2:1 target)
    risk = entry_price - stop_loss
    target = entry_price + (risk * 2)
    risk_reward = f"1:{2}" if risk > 0 else "N/A"
    risk_percent = (risk / entry_price) * 100 if entry_price > 0 else 0
    
    return {
        'symbol': symbol,
        'status': status,
        'box_top': round(latest_box.top, 2),
        'box_bottom': round(latest_box.bottom, 2),
        'entry_price': round(entry_price, 2),
        'stop_loss': round(stop_loss, 2),
        'current_price': round(current_price, 2),
        'target_2r': round(target, 2),
        'risk_percent': round(risk_percent, 2),
        'risk_reward': risk_reward,
        'volume_confirmed': latest_box.breakout_volume_confirmed if hasattr(latest_box, 'breakout_volume_confirmed') else False
    }


# Test signal generation
if test_df is not None and boxes:
    signal = generate_signals(enhanced_df, boxes, 'RELIANCE.NS')
    print("\n📊 Signal Summary:")
    for key, value in signal.items():
        print(f"  {key}: {value}")


📊 Signal Summary:
  symbol: RELIANCE.NS
  status: Post-Breakout
  box_top: 1524.7
  box_bottom: 1517.6
  entry_price: 1526.22
  stop_loss: 1517.6
  current_price: 1559.2
  target_2r: 1543.47
  risk_percent: 0.57
  risk_reward: 1:2
  volume_confirmed: False


## 5. Visualization with Plotly

In [6]:
def plot_darvas_chart(df: pd.DataFrame, boxes: List[DarvasBox], symbol: str, 
                      show_last_n_days: int = 120) -> go.Figure:
    """
    Create an interactive candlestick chart with Darvas Boxes overlay.
    
    Args:
        df: DataFrame with OHLCV data
        boxes: List of Darvas Box objects
        symbol: Stock symbol
        show_last_n_days: Number of days to display
    
    Returns:
        Plotly Figure object
    """
    # Filter to last N days for better visualization
    df_plot = df.tail(show_last_n_days).copy()
    
    # Create subplots (price + volume)
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.7, 0.3],
        subplot_titles=(f'{symbol} - Darvas Box Analysis', 'Volume')
    )
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df_plot['date'],
            open=df_plot['open'],
            high=df_plot['high'],
            low=df_plot['low'],
            close=df_plot['close'],
            name='Price',
            increasing_line_color='#26a69a',
            decreasing_line_color='#ef5350'
        ),
        row=1, col=1
    )
    
    # Add volume bars
    colors = ['#26a69a' if close >= open else '#ef5350' 
              for close, open in zip(df_plot['close'], df_plot['open'])]
    
    fig.add_trace(
        go.Bar(
            x=df_plot['date'],
            y=df_plot['volume'],
            name='Volume',
            marker_color=colors,
            opacity=0.7
        ),
        row=2, col=1
    )
    
    # Add 20-day average volume line
    fig.add_trace(
        go.Scatter(
            x=df_plot['date'],
            y=df_plot['avg_volume'],
            name='Avg Volume (20d)',
            line=dict(color='orange', width=1, dash='dash')
        ),
        row=2, col=1
    )
    
    # Add Darvas Boxes as rectangles
    box_colors = ['rgba(66, 133, 244, 0.2)', 'rgba(52, 168, 83, 0.2)', 
                  'rgba(251, 188, 4, 0.2)', 'rgba(234, 67, 53, 0.2)']
    
    plot_start_date = df_plot['date'].min()
    plot_end_date = df_plot['date'].max()
    
    for i, box in enumerate(boxes):
        # Only show boxes that overlap with our plot window
        box_end = box.end_date if box.end_date else plot_end_date
        
        if box.start_date <= plot_end_date and box_end >= plot_start_date:
            color_idx = i % len(box_colors)
            
            # Clip box dates to plot window
            x0 = max(box.start_date, plot_start_date)
            x1 = min(box_end, plot_end_date)
            
            # Add box rectangle
            fig.add_shape(
                type="rect",
                x0=x0, y0=box.bottom,
                x1=x1, y1=box.top,
                fillcolor=box_colors[color_idx],
                line=dict(color=box_colors[color_idx].replace('0.2', '0.8'), width=2),
                row=1, col=1
            )
            
            # Add box top line (entry level)
            fig.add_hline(
                y=box.top,
                line_dash="dash",
                line_color="green",
                line_width=1,
                annotation_text=f"Box Top: {box.top:.2f}",
                annotation_position="right",
                row=1, col=1
            )
            
            # Add box bottom line (stop-loss level)
            fig.add_hline(
                y=box.bottom,
                line_dash="dash",
                line_color="red",
                line_width=1,
                annotation_text=f"Stop: {box.bottom:.2f}",
                annotation_position="right",
                row=1, col=1
            )
    
    # Mark breakout points
    breakout_df = df_plot[df_plot['breakout'] == True]
    if len(breakout_df) > 0:
        fig.add_trace(
            go.Scatter(
                x=breakout_df['date'],
                y=breakout_df['high'] * 1.02,
                mode='markers+text',
                name='Breakout',
                marker=dict(symbol='triangle-up', size=15, color='lime'),
                text=['🚀' if v else '⚠️' for v in breakout_df['volume_confirmed']],
                textposition='top center'
            ),
            row=1, col=1
        )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'📊 {symbol} - Darvas Box Analysis',
            font=dict(size=20)
        ),
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=700,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    
    return fig


# Test visualization
if test_df is not None and boxes:
    fig = plot_darvas_chart(enhanced_df, boxes, 'RELIANCE.NS')
    fig.show()

## 6. Main Analysis - Enter Your Tickers Here! 🎯

In [7]:
# =====================================================
# 🎯 ENTER YOUR STOCK TICKERS HERE
# =====================================================
# Use .NS suffix for NSE stocks
# Use .BO suffix for BSE stocks

STOCK_LIST = [
    'RELIANCE.NS',
    'TCS.NS',
    'HDFCBANK.NS',
    'INFY.NS',
    'ICICIBANK.NS',
    'BHARTIARTL.NS',
    'SBIN.NS',
    'WIPRO.NS',
    'TATAMOTORS.NS',
    'AXISBANK.NS'
]

print(f"🔍 Analyzing {len(STOCK_LIST)} stocks...")

🔍 Analyzing 10 stocks...


In [8]:
def analyze_stocks(stock_list: List[str]) -> Tuple[pd.DataFrame, Dict]:
    """
    Run Darvas Box analysis on a list of stocks.
    
    Args:
        stock_list: List of stock symbols
    
    Returns:
        Tuple of (summary DataFrame, dictionary of results)
    """
    results = []
    all_data = {}
    
    for symbol in stock_list:
        print(f"\n{'='*50}")
        print(f"Analyzing: {symbol}")
        print('='*50)
        
        # Fetch data
        df = fetch_stock_data(symbol, LOOKBACK_PERIOD)
        
        if df is None or len(df) < 50:
            results.append({
                'symbol': symbol,
                'status': 'No Data',
                'box_top': None,
                'box_bottom': None,
                'entry_price': None,
                'stop_loss': None,
                'current_price': None,
                'risk_percent': None,
                'risk_reward': None
            })
            continue
        
        # Detect boxes
        boxes, enhanced_df = detect_darvas_boxes(
            df, 
            confirmation_days=CONFIRMATION_DAYS,
            volume_multiplier=VOLUME_MULTIPLIER
        )
        
        print(f"📦 Found {len(boxes)} Darvas Boxes")
        
        # Generate signals
        signal = generate_signals(enhanced_df, boxes, symbol)
        results.append(signal)
        
        # Store data for visualization
        all_data[symbol] = {
            'df': enhanced_df,
            'boxes': boxes,
            'signal': signal
        }
        
        print(f"Status: {signal['status']}")
        if signal['entry_price']:
            print(f"Entry: ₹{signal['entry_price']} | Stop: ₹{signal['stop_loss']}")
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(results)
    
    return summary_df, all_data


# Run the analysis
import time
start_time = time.time()

summary_df, all_data = analyze_stocks(STOCK_LIST)

elapsed_time = time.time() - start_time
print(f"\n\n⏱️ Analysis completed in {elapsed_time:.2f} seconds")


Analyzing: RELIANCE.NS
✅ Loaded 251 days of data for RELIANCE.NS
📦 Found 8 Darvas Boxes
Status: Post-Breakout
Entry: ₹1526.22 | Stop: ₹1517.6

Analyzing: TCS.NS
✅ Loaded 251 days of data for TCS.NS
📦 Found 3 Darvas Boxes
Status: Box Closed
Entry: ₹3991.83 | Stop: ₹3402.26

Analyzing: HDFCBANK.NS
✅ Loaded 251 days of data for HDFCBANK.NS
📦 Found 9 Darvas Boxes
Status: Inside Box
Entry: ₹1014.31 | Stop: ₹940.0

Analyzing: INFY.NS
✅ Loaded 251 days of data for INFY.NS
📦 Found 1 Darvas Boxes
Status: Box Closed
Entry: ₹1870.71 | Stop: ₹1741.51

Analyzing: ICICIBANK.NS
✅ Loaded 251 days of data for ICICIBANK.NS
📦 Found 10 Darvas Boxes
Status: Inside Box
Entry: ₹1446.44 | Stop: ₹1317.4

Analyzing: BHARTIARTL.NS
✅ Loaded 251 days of data for BHARTIARTL.NS
📦 Found 8 Darvas Boxes
Status: Box Closed
Entry: ₹2176.67 | Stop: ₹2074.1

Analyzing: SBIN.NS
✅ Loaded 251 days of data for SBIN.NS
📦 Found 10 Darvas Boxes
Status: Inside Box
Entry: ₹1000.0 | Stop: ₹944.45

Analyzing: WIPRO.NS
✅ Loaded 251 d

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TATAMOTORS.NS"}}}
$TATAMOTORS.NS: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


⚠️ No data found for TATAMOTORS.NS. Check if the ticker is valid.

Analyzing: AXISBANK.NS
✅ Loaded 251 days of data for AXISBANK.NS
📦 Found 6 Darvas Boxes
Status: Box Closed
Entry: ₹1305.3 | Stop: ₹1255.0


⏱️ Analysis completed in 3.61 seconds


## 7. Summary Output Table 📋

In [9]:
# Display styled summary table
def style_status(val):
    """Color code the status column."""
    if pd.isna(val):
        return ''
    if 'Breakout' in str(val) and 'Volume ✓' in str(val):
        return 'background-color: #1b5e20; color: white'
    elif 'Breakout' in str(val):
        return 'background-color: #33691e; color: white'
    elif 'Inside Box' in str(val):
        return 'background-color: #0d47a1; color: white'
    elif 'No Setup' in str(val) or 'No Data' in str(val):
        return 'background-color: #424242; color: white'
    return ''

# Select and reorder columns for display
display_cols = ['symbol', 'status', 'current_price', 'box_top', 'box_bottom', 
                'entry_price', 'stop_loss', 'risk_percent', 'risk_reward']

display_df = summary_df[[col for col in display_cols if col in summary_df.columns]].copy()

# Rename columns for better readability
display_df.columns = ['Symbol', 'Status', 'Current Price', 'Box Top', 'Box Bottom',
                      'Entry Price', 'Stop Loss', 'Risk %', 'Risk:Reward']

print("\n" + "="*80)
print("📊 DARVAS BOX ANALYSIS SUMMARY")
print("="*80 + "\n")

# Apply styling and display
styled_df = display_df.style.applymap(style_status, subset=['Status'])
styled_df = styled_df.format({
    'Current Price': '₹{:.2f}',
    'Box Top': '₹{:.2f}',
    'Box Bottom': '₹{:.2f}',
    'Entry Price': '₹{:.2f}',
    'Stop Loss': '₹{:.2f}',
    'Risk %': '{:.2f}%'
}, na_rep='-')

display(styled_df)


📊 DARVAS BOX ANALYSIS SUMMARY



,Symbol,Status,Current Price,Box Top,Box Bottom,Entry Price,Stop Loss,Risk %,Risk:Reward
0,RELIANCE.NS,Post-Breakout,₹1558.40,₹1524.70,₹1517.60,₹1526.22,₹1517.60,0.57%,1:2
1,TCS.NS,Box Closed,₹3289.10,₹3987.84,₹3402.26,₹3991.83,₹3402.26,14.77%,1:2
2,HDFCBANK.NS,Inside Box,₹995.20,₹1013.30,₹940.00,₹1014.31,₹940.00,7.33%,1:2
3,INFY.NS,Box Closed,₹1659.00,₹1868.84,₹1741.51,₹1870.71,₹1741.51,6.91%,1:2
4,ICICIBANK.NS,Inside Box,₹1356.40,₹1445.00,₹1317.40,₹1446.44,₹1317.40,8.92%,1:2
5,BHARTIARTL.NS,Box Closed,₹2108.50,₹2174.50,₹2074.10,₹2176.67,₹2074.10,4.71%,1:2
6,SBIN.NS,Inside Box,₹966.75,₹999.00,₹944.45,₹1000.00,₹944.45,5.55%,1:2
7,WIPRO.NS,Box Closed,₹268.01,₹314.81,₹296.17,₹315.12,₹296.17,6.01%,1:2
8,TATAMOTORS.NS,No Data,-,-,-,-,-,-,-
9,AXISBANK.NS,Box Closed,₹1228.00,₹1304.00,₹1255.00,₹1305.30,₹1255.00,3.85%,1:2


In [ ]:
# Filter and display actionable setups
actionable = summary_df[
    (summary_df['status'].str.contains('Inside Box|Breakout', na=False))
].copy()

if len(actionable) > 0:
    print("\n🎯 ACTIONABLE SETUPS")
    print("-" * 50)
    for _, row in actionable.iterrows():
        emoji = '🟢' if 'Breakout' in row['status'] else '🔵'
        print(f"{emoji} {row['symbol']}: {row['status']}")
        print(f"   Entry: ₹{row['entry_price']} | Stop: ₹{row['stop_loss']} | Risk: {row['risk_percent']:.2f}%")
        print()
else:
    print("\n⚪ No actionable setups found in the current scan.")

## 8. Visualize Individual Stocks 📈

In [10]:
# Generate charts for stocks with active setups
for symbol, data in all_data.items():
    if data['boxes']:  # Only show if boxes were detected
        print(f"\n📊 Chart for {symbol}:")
        fig = plot_darvas_chart(data['df'], data['boxes'], symbol)
        fig.show()


📊 Chart for RELIANCE.NS:



📊 Chart for TCS.NS:



📊 Chart for HDFCBANK.NS:



📊 Chart for INFY.NS:



📊 Chart for ICICIBANK.NS:



📊 Chart for BHARTIARTL.NS:



📊 Chart for SBIN.NS:



📊 Chart for WIPRO.NS:



📊 Chart for AXISBANK.NS:


## 9. Quick Analysis - Single Stock 🔍

In [12]:
def quick_analyze(symbol: str):
    """
    Quickly analyze a single stock and display results.
    
    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
    """
    print(f"\n🔍 Quick Analysis: {symbol}")
    print("="*50)
    
    # Fetch and analyze
    df = fetch_stock_data(symbol)
    if df is None:
        return
    
    boxes, enhanced_df = detect_darvas_boxes(df)
    signal = generate_signals(enhanced_df, boxes, symbol)
    
    # Print signal
    print(f"\n📊 Signal Summary:")
    print(f"  Status: {signal['status']}")
    print(f"  Current Price: ₹{signal['current_price']}")
    if signal['entry_price']:
        print(f"  Box Range: ₹{signal['box_bottom']} - ₹{signal['box_top']}")
        print(f"  Entry Price: ₹{signal['entry_price']}")
        print(f"  Stop Loss: ₹{signal['stop_loss']}")
        print(f"  Target (2R): ₹{signal.get('target_2r', 'N/A')}")
        print(f"  Risk: {signal['risk_percent']:.2f}%")
    
    # Show chart
    if boxes:
        fig = plot_darvas_chart(enhanced_df, boxes, symbol)
        fig.show()
    else:
        print("\n⚠️ No Darvas Boxes detected for this stock.")


# Example usage:
# quick_analyze('TATASTEEL.NS')

In [13]:
# 🎯 Enter a single stock for quick analysis
quick_analyze('BAJFINANCE.NS')


🔍 Quick Analysis: BAJFINANCE.NS
✅ Loaded 251 days of data for BAJFINANCE.NS

📊 Signal Summary:
  Status: Inside Box
  Current Price: ₹1005.4
  Box Range: ₹995.0 - ₹1061.0
  Entry Price: ₹1062.06
  Stop Loss: ₹995.0
  Target (2R): ₹1196.18
  Risk: 6.31%


---

## 📝 Notes & Tips

### Understanding the Signals:
- **Inside Box**: Stock is consolidating within a valid Darvas Box. Watch for breakout.
- **Breakout (Volume ✓)**: Stock has broken above the box top with strong volume. Consider entry.
- **Breakout (Low Volume)**: Breakout occurred but volume wasn't confirming. Higher risk.
- **No Setup**: Stock is not near 52-week high or no valid box has formed.

### Trading Guidelines:
1. **Entry**: Enter on a daily close above Box Top with volume confirmation
2. **Stop-Loss**: Place initial stop at Box Bottom
3. **Trailing Stop**: As new higher boxes form, move stop to new Box Bottom
4. **Position Sizing**: Risk only 1-2% of capital per trade

### Disclaimer:
⚠️ This tool is for educational purposes only. Always do your own research and consult a financial advisor before making investment decisions.

---

*Built with ❤️ for Indian Markets*